In [ ]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import re

In [ ]:
data=pd.read_excel('赛题1数据材料/赛题1数据集/一般纳税人.xlsx',index_col='企业编号')

In [ ]:
data.columns

In [ ]:
data.drop_duplicates(keep='first',inplace=True)
data

In [ ]:
data_1=pd.get_dummies(data['纳税人资格'],prefix='纳税人资格')

In [ ]:
data_1=data_1.groupby('企业编号').sum().T

In [ ]:
data_1

In [ ]:
#规整日期格式
def convert(cell):
    pattern=r'\d+'
    if cell == '--' or len(re.findall(pattern,cell)) == 0:
        return np.nan
    return '-'.join(re.findall(pattern, cell))

In [ ]:
datatime=data[['认定日期']].applymap(str).applymap(convert)
datatime

In [ ]:
#计算时间间隔
def to_datetime(cell):
    if not isinstance(cell, str):
        return cell
    try:
        return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days
    except:
        print(cell)

In [ ]:
data_2=datatime.applymap(to_datetime).groupby('企业编号').mean().T.reindex(columns=list(range(1001,4001)))
data_2

In [ ]:
data_3=data[['有效日期期起']].applymap(to_datetime).groupby('企业编号').mean().T.reindex(columns=list(range(1001,4001)))
data_3

In [ ]:
data_4=data[['有效截止日期']].applymap(to_datetime).groupby('企业编号').mean().T.reindex(columns=list(range(1001,4001)))
data_4

In [ ]:
def limit(cell):
    if cell > 1:
        return 1
    return cell

In [ ]:
data_5=pd.get_dummies(data['登记注册类型'],prefix='登记注册类型').groupby('企业编号').sum().T.applymap(limit).reindex(columns=list(range(1001,4001))).fillna(0)
data_5

In [ ]:
data_6=pd.get_dummies(data['纳税人状态'],prefix='纳税人状态').groupby('企业编号').mean().T.reindex(columns=list(range(1001,4001))).fillna(0)
data_6

In [ ]:
data_7=pd.get_dummies(data['扣缴义务'],prefix='扣缴义务').groupby('企业编号').mean().T.reindex(columns=list(range(1001,4001))).fillna(0)
data_7

In [ ]:
data_8=pd.get_dummies(data['出口状态备案状态'],prefix='出口状态备案状态').groupby('企业编号').mean().T.reindex(columns=list(range(1001,4001))).fillna(0)
data_8

In [ ]:
data=pd.concat([data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8])
data.to_pickle('一般纳税人.pickle')
data

In [ ]:
data.index